# Social Bot
**Goal**: correctly identify whether a response is socially acceptable

In [206]:
import tensorflow as tf
import numpy
from numpy import ndarray

## Define constants

In [207]:
sample_count = {
    "training": 8000,
    "test": 1300,
    "validation": 2000
    }

category_min = 1
category_max = 5

label_type_count = category_max - category_min

image_height = 1
image_width = 2

flat_size = image_height * image_width

label_type_count = 2 # number of distinct labels (social or not)

learning_rate = 0.5

## Auxiliary functions

### Show result

In [208]:
def show_result(x):
    """Print the result"""
    print("{:.0f}%".format(x * 100))

In [209]:
numpy.exp([-1, -2, -3])

array([ 0.36787944,  0.13533528,  0.04978707])

### Automaticaly generate data sets

In [210]:
def social_rule(xs: ndarray, threshold=0.2):
    """
    The rule for juding whether a response is socially polite
    Args:
        xs (ndarray): inputs and responses (2D array)
        threshold (float): threshold above which it is considerred socially acceptible
    """
    def one_hot(x):
        if x:
            return [1, 0]
        else:
            return [0, 1]
    return one_hot(numpy.exp(-numpy.absolute(numpy.diff(xs))) > threshold)


In [211]:
def social_samples(n: int, low: int, high: int):
    """
    Args:
        n (int): number of data points
        low (int): lower bound
        high (int): upper bound
    """
    def aux(_):
        data = numpy.random.randint(low, high=high, size=2)
        return numpy.concatenate((data, social_rule(data)))
    data = numpy.vstack(map(aux, range(n)))
    print(data)
    return {
        "data": data[:, 0:2],
        "labels": data[:, 2:]
    }


In [212]:
social_samples(10, 1, 5)

[[4 1 0 1]
 [3 4 1 0]
 [1 1 1 0]
 [2 1 1 0]
 [3 1 0 1]
 [1 2 1 0]
 [2 4 0 1]
 [4 1 0 1]
 [1 3 0 1]
 [3 3 1 0]]


{'data': array([[4, 1],
        [3, 4],
        [1, 1],
        [2, 1],
        [3, 1],
        [1, 2],
        [2, 4],
        [4, 1],
        [1, 3],
        [3, 3]]), 'labels': array([[0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0]])}

# Reflection-type testing data

In [213]:
low, high = category_min, category_max
dataset = {
    "training": social_samples(sample_count['training'], low, high),
    "test": social_samples(sample_count['test'], low, high),
    "validation": social_samples(sample_count['validation'], low, high)
}

[[3 2 1 0]
 [1 2 1 0]
 [3 3 1 0]
 ..., 
 [2 4 0 1]
 [2 2 1 0]
 [1 2 1 0]]
[[3 3 1 0]
 [3 1 0 1]
 [2 1 1 0]
 ..., 
 [2 4 0 1]
 [1 1 1 0]
 [4 3 1 0]]
[[2 2 1 0]
 [4 2 0 1]
 [1 2 1 0]
 ..., 
 [2 2 1 0]
 [4 4 1 0]
 [3 3 1 0]]


In [214]:
dataset['training']['data'].shape

(8000, 2)

In [215]:
dataset['test']['labels'][1]

array([0, 1])

# Memory allocation

In [216]:
x = tf.placeholder(tf.float32, [None, flat_size])

In [217]:
W = tf.Variable(tf.zeros([flat_size, label_type_count]))

In [218]:
b = tf.Variable(tf.zeros([label_type_count]))

In [219]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [220]:
y_ = tf.placeholder(tf.float32, [None, label_type_count])

In [221]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
)

In [222]:
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [223]:
# create a session
session = tf.InteractiveSession()

In [224]:
tf.global_variables_initializer().run()

# I. Regular Neural Net

## Training



In [225]:
session.run(train_step, feed_dict={x: dataset['training']['data'], y_: dataset['training']['labels']})

In [226]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

In [227]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Test

In [228]:
test_result = session.run(accuracy, feed_dict={x: dataset['test']['data'], y_: dataset['test']['labels']})

In [229]:
show_result(test_result)

61%


## Validation

In [230]:
exam_result = session.run(accuracy, feed_dict={x: dataset['validation']['data'], y_: dataset['validation']['labels']})

In [231]:
show_result(exam_result)

61%
